In [17]:
import numpy as np
import keras
import matplotlib.pyplot as plt
import re
import pandas as pd
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from string import ascii_lowercase

# Importing data and Cleaning it.

In [2]:
data=pd.read_csv('E:\\Shreyans\\Data sets\\Hindi\\NER Hindi English Code Mixed Tweets.tsv',delimiter='\t',encoding='utf-8')

In [3]:
print(list(data.columns.values)) #file header

['Gully', 'en', 'O']


In [4]:
data.head(5)

,Gully,en,O
0,cricket,en,O
1,chal,hi,O
2,raha,hi,O
3,hain,hi,O
4,yaha,hi,O


In [5]:
data = data.drop(['O'], axis=1)

In [6]:
data = data.rename(index=str, columns={'Gully':'Word', 'en':'label'})
data=data.reset_index(drop=True)

In [7]:
l=[]
for i in range(0,len(data.label)):
    a=data.label[i]
    if a=='rest':
        l.append(i)

In [8]:
data=data.drop(l)
data=data.reset_index(drop=True)

In [9]:
data.head(10)

,Word,label
0,cricket,en
1,chal,hi
2,raha,hi
3,hain,hi
4,yaha,hi
5,I,en
6,liked,en
7,a,en
8,@YouTube,en
9,video,en


In [10]:
l1=[]
for i in range(0, len(data.Word)):
    match=re.search(r'https?:\/\/.*[\r\n]*', data.Word[i])
    if match:
        l1.append(i)
        

In [11]:
data=data.drop(l1)
data=data.reset_index(drop=True)

In [12]:
data.head(10)

,Word,label
0,cricket,en
1,chal,hi
2,raha,hi
3,hain,hi
4,yaha,hi
5,I,en
6,liked,en
7,a,en
8,@YouTube,en
9,video,en


In [13]:
len(data)

12749

In [14]:
data.to_csv('New_Hinglish_Classification.csv',encoding='utf-8')

In [3]:
data = pd.read_csv('E:\\Shreyans\\Data sets\\Hindi\\New_Hinglish_Classification_final.csv',encoding='utf-8')

In [8]:
data = data.iloc[:,1:]


# Converting Values to vector

In [9]:
x_data=data.iloc[:,0:1]
y_data=data.iloc[:,1:2]

In [10]:
x_data.head()

,Word
0,cricket
1,chal
2,raha
3,hain
4,yaha


In [11]:
y_data.head()

,label
0,en
1,hi
2,hi
3,hi
4,hi


In [18]:
labelencoder = LabelEncoder()
#x_data_vec=labelencoder.fit_transform(x_data)
y_data_vec = labelencoder.fit_transform(y_data)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:129: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [19]:
x_data_vec

array([3371, 3241, 4985, ...,  829,  127, 2033], dtype=int64)

In [20]:
y_data_vec

array([0, 1, 1, ..., 0, 0, 0], dtype=int64)

In [126]:
y_data_vec = keras.utils.to_categorical(y_data_vec, 2)

In [69]:
sc = MinMaxScaler(feature_range = (0, 1))
x_data_vec = sc.fit_transform(x_data_vec)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:429: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, _DataConversionWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:321: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:356: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)


# Splitting of data

In [21]:
X_train, X_test, Y_train, Y_test = train_test_split(x_data_vec, y_data_vec, test_size=0.20)

In [36]:
X_train=X_train.reshape(10199,1,1)
X_test=X_test.reshape(2550,1,1)
Y_train=Y_train.reshape(10199,1)
Y_test=Y_test.reshape(2550,1)

In [37]:
print('x_train shape:', X_train.shape)
print('x_test shape:', X_test.shape)
print('Y_train shape:', Y_train.shape)
print('Y_test shape:', Y_test.shape)

x_train shape: (10199, 1, 1)
x_test shape: (2550, 1, 1)
Y_train shape: (10199, 1)
Y_test shape: (2550, 1)


# Model Creation

In [38]:
model_classifier =Sequential()
model_classifier.add(LSTM(128, return_sequences = True, input_shape = (X_train.shape[1], 1)))
model_classifier.add(LSTM(64, activation='tanh'))
model_classifier.add(Dense(1, activation='sigmoid'))
#model_classifier.add(Dense(2, activation='softmax'))
model_classifier.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_7 (LSTM)                (None, 1, 128)            66560     
_________________________________________________________________
lstm_8 (LSTM)                (None, 64)                49408     
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 65        
Total params: 116,033
Trainable params: 116,033
Non-trainable params: 0
_________________________________________________________________


In [39]:
# try using different optimizers and different optimizer configs
model_classifier.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [40]:
# Write the training input and output, batch size, and testing input and output

model_classifier.fit(X_train, Y_train, 
          batch_size=128, 
          epochs=50, 
          validation_data=(X_test, Y_test))

Train on 10199 samples, validate on 2550 samples
Epoch 1/50
10199/10199 [==============================] - 12s 1ms/step - loss: 0.6853 - acc: 0.5415 - val_loss: 0.6816 - val_acc: 0.5424
Epoch 2/50
10199/10199 [==============================] - 2s 160us/step - loss: 0.6771 - acc: 0.5411 - val_loss: 0.6825 - val_acc: 0.5424
Epoch 3/50
10199/10199 [==============================] - 2s 156us/step - loss: 0.6782 - acc: 0.5345 - val_loss: 0.6813 - val_acc: 0.5424
Epoch 4/50
10199/10199 [==============================] - 2s 171us/step - loss: 0.6770 - acc: 0.5436 - val_loss: 0.6807 - val_acc: 0.5424
Epoch 5/50
10199/10199 [==============================] - 2s 163us/step - loss: 0.6776 - acc: 0.5411 - val_loss: 0.6837 - val_acc: 0.4961
Epoch 6/50
10199/10199 [==============================] - 1s 137us/step - loss: 0.6784 - acc: 0.5361 - val_loss: 0.6858 - val_acc: 0.5427
Epoch 7/50
10199/10199 [==============================] - 1s 145us/step - loss: 0.6781 - acc: 0.5430 - val_loss: 0.6873 - va

In [134]:
# try using different optimizers and different optimizer configs
model_classifier.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [140]:
score, acc = model_classifier.evaluate(X_test, Y_test, batch_size=128)
print('Test score:', score)
print('Test accuracy:', acc)

2550/2550 [==============================] - 0s 26us/step
Test score: 0.645162695342
Test accuracy: 0.623921568253


In [149]:
model_classifier_ann = Sequential()

# Adding the input layer and the first hidden layer
model_classifier_ann.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim=1))

# Adding the second hidden layer
model_classifier_ann.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))

# Adding the output layer
model_classifier_ann.add(Dense(units = 2, kernel_initializer = 'uniform', activation = 'softmax'))

# Compiling the ANN
model_classifier_ann.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

# Fitting the ANN to the Training set
model_classifier_ann.fit(X_train, Y_train, batch_size = 32, epochs = 100)

Epoch 1/100
10199/10199 [==============================] - 3s 278us/step - loss: 0.6905 - acc: 0.5432
Epoch 2/100
10199/10199 [==============================] - 0s 36us/step - loss: 0.6895 - acc: 0.5432
Epoch 3/100
10199/10199 [==============================] - 0s 32us/step - loss: 0.6882 - acc: 0.5432
Epoch 4/100
10199/10199 [==============================] - 0s 34us/step - loss: 0.6834 - acc: 0.5432
Epoch 5/100
10199/10199 [==============================] - 0s 34us/step - loss: 0.6801 - acc: 0.5416
Epoch 6/100
10199/10199 [==============================] - 0s 35us/step - loss: 0.6791 - acc: 0.5666
Epoch 7/100
10199/10199 [==============================] - 0s 32us/step - loss: 0.6788 - acc: 0.5680: 0s - loss: 0.6774 - acc: 0.57
Epoch 8/100
10199/10199 [==============================] - 0s 29us/step - loss: 0.6788 - acc: 0.5725
Epoch 9/100
10199/10199 [==============================] - 0s 33us/step - loss: 0.6787 - acc: 0.5717
Epoch 10/100
10199/10199 [==============================] -

# Char By Char Trial

In [3]:
data = pd.read_csv('E:\\Shreyans\\Data sets\\Hindi\\New_Hinglish_Classification_final.csv',encoding='utf-8')

In [8]:
data = data.iloc[:,1:]


### Converting Values to vector

In [55]:
x_data=data.iloc[:,0:1]
y_data=data.iloc[:,1:2]

In [56]:
x_data.head()

,Word
0,cricket
1,chal
2,raha
3,hain
4,yaha


In [57]:
y_data.head()

,label
0,en
1,hi
2,hi
3,hi
4,hi


In [33]:
labelencoder = LabelEncoder()
#x_data_vec=labelencoder.fit_transform(x_data)
y_data_vec = labelencoder.fit_transform(y_data)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:129: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [58]:
y_data_vec

array([0, 1, 1, ..., 0, 0, 0], dtype=int64)

In [68]:
od=dict((ch,idx) for idx, ch in enumerate(ascii_lowercase,1))

In [69]:
od

{'a': 1,
 'b': 2,
 'c': 3,
 'd': 4,
 'e': 5,
 'f': 6,
 'g': 7,
 'h': 8,
 'i': 9,
 'j': 10,
 'k': 11,
 'l': 12,
 'm': 13,
 'n': 14,
 'o': 15,
 'p': 16,
 'q': 17,
 'r': 18,
 's': 19,
 't': 20,
 'u': 21,
 'v': 22,
 'w': 23,
 'x': 24,
 'y': 25,
 'z': 26}

In [26]:
data=pd.read_csv("E:\\Shreyans\\Data sets\\Hindi\\New_Hinglish_Classification_final_1.csv")

data=data.iloc[:,1:]


In [70]:
data_vec=[] 
for i in range(0,len(data.Word)):
    
    data_vec.append(data.Word[i])

    
data_vec1=[]
for i in range(0,len(data_vec)):
    data_vec1.append(list(data_vec[i]))
    

w=len(data_vec1)

In [71]:

for i in range(0,w):
    for j in range(0,len(data_vec1[i])):
        q=data_vec1[i][j]
        ff=od.get(q)
        data_vec1[i][j]= data_vec1[i][j].replace(q,str(ff))
    

In [72]:
for i in range(0,w):
    data_vec1[i] = list(map(int, data_vec1[i]))

In [73]:
data_vec1

[[3, 18, 9, 3, 11, 5, 20],
 [3, 8, 1, 12],
 [18, 1, 8, 1],
 [8, 1, 9, 14],
 [25, 1, 8, 1],
 [9],
 [12, 9, 11, 5, 4],
 [1],
 [25, 15, 21, 20, 21, 2, 5],
 [22, 9, 4, 5, 15],
 [11, 1, 2, 8, 9],
 [16, 1, 12, 11, 15, 14],
 [16, 5],
 [1, 1, 19, 15, 15, 14],
 [8, 1, 9],
 [11, 9, 19, 8, 15, 18, 5],
 [11, 21, 13, 1, 18],
 [22, 15, 3, 1, 12],
 [3, 15, 22, 5, 18],
 [2, 25],
 [19, 20, 5, 16, 8, 5, 14],
 [1, 2],
 [25, 5, 8],
 [11, 15, 14],
 [2, 15, 12],
 [18, 1, 8, 1],
 [8, 1, 9],
 [1, 1, 10],
 [16, 5],
 [1, 1, 16],
 [23, 1, 12, 5],
 [13, 5],
 [23, 9, 6, 9],
 [11, 5],
 [19, 1, 20, 8],
 [13, 9, 14],
 [23, 5, 5, 4],
 [2, 8, 9],
 [6, 18, 5, 5],
 [4, 5],
 [18, 1, 8, 5],
 [8, 1, 9],
 [18, 20],
 [7, 21, 18, 13, 5, 5, 20, 18, 1, 13, 18, 1, 8, 9, 13],
 [13,
  19,
  7,
  4,
  5,
  4,
  9,
  3,
  1,
  20,
  5,
  19,
  1,
  23,
  1,
  18,
  4,
  20,
  15,
  7,
  21,
  18,
  21,
  10,
  9],
 [16, 1, 16, 1],
 [25, 5, 16, 9, 5, 5, 5],
 [3, 15, 14, 7, 18, 1, 20, 21, 12, 1, 20, 9, 15, 14, 19],
 [1, 14, 4],
 [3, 5,

In [74]:
x_data_vec = np.array(data_vec1)

In [75]:
x_data_vec.shape

(12749,)

In [76]:
c=0
for i in range(0,len(data.Word)):
    a=len(data.Word[i])
    if a>c:
        c=a
        cc=data.Word[i]

In [128]:
data_vec1

[[3, 18, 9, 3, 11, 5, 20],
 [3, 8, 1, 12],
 [18, 1, 8, 1],
 [8, 1, 9, 14],
 [25, 1, 8, 1],
 [9],
 [12, 9, 11, 5, 4],
 [1],
 [25, 15, 21, 20, 21, 2, 5],
 [22, 9, 4, 5, 15],
 [11, 1, 2, 8, 9],
 [16, 1, 12, 11, 15, 14],
 [16, 5],
 [1, 1, 19, 15, 15, 14],
 [8, 1, 9],
 [11, 9, 19, 8, 15, 18, 5],
 [11, 21, 13, 1, 18],
 [22, 15, 3, 1, 12],
 [3, 15, 22, 5, 18],
 [2, 25],
 [19, 20, 5, 16, 8, 5, 14],
 [1, 2],
 [25, 5, 8],
 [11, 15, 14],
 [2, 15, 12],
 [18, 1, 8, 1],
 [8, 1, 9],
 [1, 1, 10],
 [16, 5],
 [1, 1, 16],
 [23, 1, 12, 5],
 [13, 5],
 [23, 9, 6, 9],
 [11, 5],
 [19, 1, 20, 8],
 [13, 9, 14],
 [23, 5, 5, 4],
 [2, 8, 9],
 [6, 18, 5, 5],
 [4, 5],
 [18, 1, 8, 5],
 [8, 1, 9],
 [18, 20],
 [7, 21, 18, 13, 5, 5, 20, 18, 1, 13, 18, 1, 8, 9, 13],
 [13,
  19,
  7,
  4,
  5,
  4,
  9,
  3,
  1,
  20,
  5,
  19,
  1,
  23,
  1,
  18,
  4,
  20,
  15,
  7,
  21,
  18,
  21,
  10,
  9],
 [16, 1, 16, 1],
 [25, 5, 16, 9, 5, 5, 5],
 [3, 15, 14, 7, 18, 1, 20, 21, 12, 1, 20, 9, 15, 14, 19],
 [1, 14, 4],
 [3, 5,

In [1]:
len('yandasmedyalimonsatonurluyasa')

29

In [79]:
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [80]:
maxl=30
x_data_vec = sequence.pad_sequences(data_vec1, maxlen=maxl)

In [144]:
data_vec1

[[3, 18, 9, 3, 11, 5, 20],
 [3, 8, 1, 12],
 [18, 1, 8, 1],
 [8, 1, 9, 14],
 [25, 1, 8, 1],
 [9],
 [12, 9, 11, 5, 4],
 [1],
 [25, 15, 21, 20, 21, 2, 5],
 [22, 9, 4, 5, 15],
 [11, 1, 2, 8, 9],
 [16, 1, 12, 11, 15, 14],
 [16, 5],
 [1, 1, 19, 15, 15, 14],
 [8, 1, 9],
 [11, 9, 19, 8, 15, 18, 5],
 [11, 21, 13, 1, 18],
 [22, 15, 3, 1, 12],
 [3, 15, 22, 5, 18],
 [2, 25],
 [19, 20, 5, 16, 8, 5, 14],
 [1, 2],
 [25, 5, 8],
 [11, 15, 14],
 [2, 15, 12],
 [18, 1, 8, 1],
 [8, 1, 9],
 [1, 1, 10],
 [16, 5],
 [1, 1, 16],
 [23, 1, 12, 5],
 [13, 5],
 [23, 9, 6, 9],
 [11, 5],
 [19, 1, 20, 8],
 [13, 9, 14],
 [23, 5, 5, 4],
 [2, 8, 9],
 [6, 18, 5, 5],
 [4, 5],
 [18, 1, 8, 5],
 [8, 1, 9],
 [18, 20],
 [7, 21, 18, 13, 5, 5, 20, 18, 1, 13, 18, 1, 8, 9, 13],
 [13,
  19,
  7,
  4,
  5,
  4,
  9,
  3,
  1,
  20,
  5,
  19,
  1,
  23,
  1,
  18,
  4,
  20,
  15,
  7,
  21,
  18,
  21,
  10,
  9],
 [16, 1, 16, 1],
 [25, 5, 16, 9, 5, 5, 5],
 [3, 15, 14, 7, 18, 1, 20, 21, 12, 1, 20, 9, 15, 14, 19],
 [1, 14, 4],
 [3, 5,

In [82]:
x_data_vec.shape

(12749, 30)

In [83]:
X_train, X_test, Y_train, Y_test = train_test_split(x_data_vec, y_data_vec, test_size=0.20)

In [91]:
X_train=X_train.reshape(10199,30,1)
X_test=X_test.reshape(2550,30,1)
Y_train=Y_train.reshape(10199,1)
Y_test=Y_test.reshape(2550,1)

In [92]:
print('x_train shape:', X_train.shape)
print('x_test shape:', X_test.shape)
print('Y_train shape:', Y_train.shape)
print('Y_test shape:', Y_test.shape)

x_train shape: (10199, 30, 1)
x_test shape: (2550, 30, 1)
Y_train shape: (10199, 1)
Y_test shape: (2550, 1)


# Model Creation

In [93]:
model_classifier =Sequential()
model_classifier.add(LSTM(128, return_sequences = True, input_shape = (X_train.shape[1], 1)))
model_classifier.add(LSTM(64, activation='tanh'))
model_classifier.add(Dense(1, activation='sigmoid'))
#model_classifier.add(Dense(2, activation='softmax'))
model_classifier.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 30, 128)           66560     
_________________________________________________________________
lstm_4 (LSTM)                (None, 64)                49408     
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 65        
Total params: 116,033
Trainable params: 116,033
Non-trainable params: 0
_________________________________________________________________


In [94]:
# try using different optimizers and different optimizer configs
model_classifier.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [95]:
# Write the training input and output, batch size, and testing input and output

model_classifier.fit(X_train, Y_train, 
          batch_size=128, 
          epochs=50, 
          validation_data=(X_test, Y_test))

Train on 10199 samples, validate on 2550 samples
Epoch 1/50
10199/10199 [==============================] - 10s 980us/step - loss: 0.5941 - acc: 0.6777 - val_loss: 0.5674 - val_acc: 0.7090
Epoch 2/50
10199/10199 [==============================] - 8s 738us/step - loss: 0.5583 - acc: 0.7169 - val_loss: 0.5532 - val_acc: 0.7227
Epoch 3/50
10199/10199 [==============================] - 8s 788us/step - loss: 0.5377 - acc: 0.7246 - val_loss: 0.5346 - val_acc: 0.7329
Epoch 4/50
10199/10199 [==============================] - 8s 743us/step - loss: 0.5229 - acc: 0.7349 - val_loss: 0.5196 - val_acc: 0.7329
Epoch 5/50
10199/10199 [==============================] - 8s 751us/step - loss: 0.5086 - acc: 0.7424 - val_loss: 0.5068 - val_acc: 0.7478
Epoch 6/50
10199/10199 [==============================] - 8s 757us/step - loss: 0.4950 - acc: 0.7533 - val_loss: 0.5056 - val_acc: 0.7353
Epoch 7/50
10199/10199 [==============================] - 8s 752us/step - loss: 0.4860 - acc: 0.7570 - val_loss: 0.4845 - 

In [101]:
score, acc = model_classifier.evaluate(X_test, Y_test, batch_size=128)
print('Test score:', score)
print('Test accuracy:', acc)

2550/2550 [==============================] - 1s 435us/step
Test score: 0.690735682319
Test accuracy: 0.519999999486


In [218]:
model_classifier2 =Sequential()
model_classifier2.add(LSTM(128, return_sequences = True, input_shape = (X_train.shape[1], 1)))
model_classifier2.add(LSTM(128, activation='relu'))
model_classifier2.add(Dense(1, activation='sigmoid'))
#model_classifier.add(Dense(2, activation='softmax'))
model_classifier2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_11 (LSTM)               (None, 30, 128)           66560     
_________________________________________________________________
lstm_12 (LSTM)               (None, 128)               131584    
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 129       
Total params: 198,273
Trainable params: 198,273
Non-trainable params: 0
_________________________________________________________________


In [219]:
# try using different optimizers and different optimizer configs
model_classifier2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [220]:
# Write the training input and output, batch size, and testing input and output

model_classifier2.fit(X_train, Y_train, 
          batch_size=128, 
          epochs=100, 
          validation_data=(X_test, Y_test))

Train on 10199 samples, validate on 2550 samples
Epoch 1/100
10199/10199 [==============================] - 20s 2ms/step - loss: 0.5995 - acc: 0.6604 - val_loss: 0.5753 - val_acc: 0.7024
Epoch 2/100
10199/10199 [==============================] - 11s 1ms/step - loss: 0.5597 - acc: 0.7079 - val_loss: 0.5761 - val_acc: 0.7090
Epoch 3/100
10199/10199 [==============================] - 11s 1ms/step - loss: 0.5320 - acc: 0.7281 - val_loss: 0.5255 - val_acc: 0.7369
Epoch 4/100
10199/10199 [==============================] - 11s 1ms/step - loss: 0.5117 - acc: 0.7437 - val_loss: 0.5112 - val_acc: 0.7431
Epoch 5/100
10199/10199 [==============================] - 11s 1ms/step - loss: 0.4954 - acc: 0.7539 - val_loss: 0.4899 - val_acc: 0.7490
Epoch 6/100
10199/10199 [==============================] - 11s 1ms/step - loss: 0.4777 - acc: 0.7599 - val_loss: 0.4762 - val_acc: 0.7620
Epoch 7/100
10199/10199 [==============================] - 11s 1ms/step - loss: 0.4605 - acc: 0.7713 - val_loss: 0.4535 - v

In [221]:
score, acc = model_classifier2.evaluate(X_test, Y_test, batch_size=128)
print('Test score:', score)
print('Test accuracy:', acc)

2550/2550 [==============================] - 1s 346us/step
Test score: 0.421384237467
Test accuracy: 0.897254902382


In [222]:
# serialize model to JSON
model_json = model_classifier2.to_json()

# Write the file name of the model

with open("lang_class_relu_model.json", "w") as json_file:
    json_file.write(model_json)
    
# serialize weights to HDF5
# Write the file name of the weights

model_classifier2.save_weights("lang_class_relu_model_weights.h5")
print("Saved model to disk")


Saved model to disk


In [116]:
prediction = model_classifier2.predict(X_test[1:2])
print('Prediction value:',prediction[0])
print('Test Label:',Y_test[1:2])

Prediction value: [ 0.92076343]
Test Label: [[1]]


In [119]:
X_test[1:2].shape

(1, 30, 1)

In [238]:
a=input()
a=a.lower()
b=list(a)
for i in range(0,len(b)):
    qw=od.get(b[i])
    b[i]=b[i].replace(b[i],str(qw))
for i in range(0,len(b)):
    b= list(map(int,b))
for i in range(0,len(b)):
    b= list(map(int,b))
b=[b]
bb=sequence.pad_sequences(b, maxlen=30)
bb=np.array(bb)
bb=bb.reshape(1,30,1)
prediction = model_classifier2.predict(bb)
print('Prediction value:',prediction[0])
if prediction[0][0] >0.5:
    print("Hindi")
else:
    print("English")


ghar
Prediction value: [ 0.99983382]
Hindi


(1, 30)

Prediction value: [ 0.95690554]
Hinglish


2550